In [21]:
import pandas as pd
import fastparquet


In [22]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(2940, 16)

In [23]:
len(df[df.tiling_compatible == True]) 

75

In [24]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format   1203
1       failed_to_extract    647
2          cant_open_file    551
3         group_structure    191
4  tile_generation_failed    179
5        no_xy_dimensions     79
6                    None     75
7  cant_extract_variables     15


In [ ]:
df[df.incompatible_reason == 'failed_to_extract'].error_message.unique()


array(['cannot unpack non-iterable NoneType object',
       'Could not find data URL in granule metadata',
       'sequence item 0: expected str instance, NoneType found'],
      dtype=object)

In [28]:
list(df[df.incompatible_reason == 'failed_to_extract'].collection_concept_id)

['C1000000103-LARC_ASDC',
 'C1000000123-LARC_ASDC',
 'C1000000144-LARC_ASDC',
 'C1000000160-LARC_ASDC',
 'C1000000743-LARC_ASDC',
 'C1000001002-LARC_ASDC',
 'C1000001003-LARC_ASDC',
 'C1000001030-LARC_ASDC',
 'C1000001034-LARC_ASDC',
 'C1000001036-LARC_ASDC',
 'C1000001043-LARC_ASDC',
 'C1000001054-LARC_ASDC',
 'C1000001141-LARC_ASDC',
 'C1000001186-LARC_ASDC',
 'C1000001190-LARC_ASDC',
 'C1214471960-ASF',
 'C1214472978-ASF',
 'C1214473165-ASF',
 'C1233781718-GES_DISC',
 'C1237114203-GES_DISC',
 'C1237114205-GES_DISC',
 'C1237114223-GES_DISC',
 'C1237114224-GES_DISC',
 'C1238517247-GES_DISC',
 'C1239900203-GES_DISC',
 'C1251051178-GES_DISC',
 'C1251051239-GES_DISC',
 'C1251051471-GES_DISC',
 'C1251051574-GES_DISC',
 'C1251051604-GES_DISC',
 'C1251051612-GES_DISC',
 'C1251051713-GES_DISC',
 'C1251051721-GES_DISC',
 'C1251100759-GES_DISC',
 'C1251100901-GES_DISC',
 'C1251100913-GES_DISC',
 'C1251100938-GES_DISC',
 'C1251100984-GES_DISC',
 'C1251101003-GES_DISC',
 'C1251101014-GES_DISC',


In [33]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and (x.size > 0 and x[0] != None))
)]
subset_df.data_variables

27                                               [SWIR1]
29                                               [TIRS2]
34     [latitude, longitude, scan_datetime, year, mon...
37     [aerosol_optical_thickness_550_land, aerosol_o...
43     [Aerosol_Optical_Thickness_550_Expected_Uncert...
                             ...                        
623    [Delta_Time, SZA, VZA, SAz, VAz, Longitude, La...
625                                             [weight]
640    [obs_id, fov_obs_id, obs_time_tai93, obs_time_...
643    [EFLUXICE, EFLUXWTR, FRSEAICE, HFLUXICE, HFLUX...
646                    [OriginalInputFiles, XmlMetadata]
Name: data_variables, Length: 160, dtype: object

In [47]:
all_unique_vars = set().union(*subset_df.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

509


In [36]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
